In [1]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f7142676034147a415e3df15c5655aad4c6cc39f01d31321b48f89eac76f18af
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


**Ejercicio 1**

In [4]:
rdd_lenguajes = SparkContext.parallelize(['python', 'r', 'c', 'rugby', 'sql']);
rdd_lenguajes.collect()

['python', 'r', 'c', 'rugby', 'sql']

In [9]:
rdd_leng_new = rdd_lenguajes.filter(lambda x: x.startswith('r'));
rdd_leng_new.collect()

['r', 'rugby']

**Ejercicio 2**

In [35]:
rdd_numeros = SparkContext.parallelize([19,20,21,22,23,24,25,26,27,28,29,30,31],3)
rdd_num_pares = rdd_numeros.filter(lambda x: 20 <= x <= 30 and x % 2 == 0)
rdd_num_pares.collect()

[20, 22, 24, 26, 28, 30]

In [29]:
rdd_num_raiz = rdd_num_pares.map(lambda x: x ** 0.5)
rdd_num_raiz.collect()

[4.47213595499958,
 4.69041575982343,
 4.898979485566356,
 5.0990195135927845,
 5.291502622129181,
 5.477225575051661]

Para reducir las particiones de manera mas eficiente utilizaria la funcion coalesce

In [37]:
rdd_num_aumentada = rdd_numeros.repartition(20)
rdd_num_aumentada.getNumPartitions()

20

**Ejercicio 3**

In [51]:
rdd_lectura_transacciones = SparkContext.textFile("transacciones")
rdd_lectura_transacciones.collect()

['(1001, 52.3)',
 '(1005, 20.8)',
 '(1001, 10.1)',
 '(1004, 52.7)',
 '(1005, 20.7)',
 '(1002, 85.3)',
 '(1004, 20.9)']

In [53]:
rdd_transacciones_clave_valor = SparkContext.parallelize([('1001, 52.3'),
 ('1005, 20.8'),
 ('1001, 10.1'),
 ('1004, 52.7'),
 ('1005, 20.7'),
 ('1002, 85.3'),
 ('1004, 20.9')])
rdd_transacciones_clave_valor.collect()

['1001, 52.3',
 '1005, 20.8',
 '1001, 10.1',
 '1004, 52.7',
 '1005, 20.7',
 '1002, 85.3',
 '1004, 20.9']

In [54]:
def proceso(rdd_proceso):
  transaccion = rdd_proceso.map(lambda x: x.split(","))
  transaccion_clave_valor = transaccion.map(lambda x: (x[0], float(x[1])))
  monto_total = transaccion_clave_valor.reduceByKey(lambda x, y: x + y)
  return monto_total

resultado = proceso(rdd_transacciones_clave_valor)

for cuenta, rdd_monto_total in resultado.collect():
  print("Cuenta: ", cuenta, "Monto Total: ", rdd_monto_total)

Cuenta:  1002 Monto Total:  85.3
Cuenta:  1001 Monto Total:  62.4
Cuenta:  1005 Monto Total:  41.5
Cuenta:  1004 Monto Total:  73.6
